In [ ]:
# DATA PREPARATION

import pandas as pd
import numpy as np

# Load data from CSV file
try:
    df = pd.read_csv('../data/synthetic/synthetic_network.csv')
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
    exit()
except pd.errors.EmptyDataError:
    print("Error: The file is empty or invalid.")
    exit()


# Identify unique nodes and create a mapping for indices
nodos = sorted(set(df['origin']).union(set(df['destination'])))
node_to_index = {node: idx for idx, node in enumerate(nodos)}
n = len(nodos)

# Initialize the cost matrix with infinity
cost_matrix = np.full((n, n), np.inf)


# Set diagonal to 0 (self-costs)
np.fill_diagonal(cost_matrix, 0)

# Fill the cost matrix with the values from the CSV
for _, row in df.iterrows():
    try:
        origen = row['origin']
        destino = row['destination']
        costo = float(row['weight'])
        cost_matrix[node_to_index[origen], node_to_index[destino]] = costo
    except KeyError:
        print("Error: Missing columns 'origin', 'destination', or 'weight'.")
        exit()
    except ValueError:
        print(f"Error: Invalid cost value on row {_}.")
        exit()

# Display the cost matrix
print("Cost Matrix:")
print(cost_matrix)

np.save('../data/synthetic/cost_matrix.npy', cost_matrix)




Offline training to learn graph properties without source/destination nodes

In [ ]:
import tensorflow as tf
C = cost_matrix
n = C.shape[0]  # Número de nodos
print("Numero de nodos")
print (n)
# Vectorizar la matriz de costes
C_flat = C.flatten()
C_flat[np.isinf(C_flat)] = 1e6  # Reemplazar infinito por un valor grande
flattened_cost_matrix = C_flat
num_nodes = n

def offline_loss(y_true, y_pred, num_nodes, flattened_cost_matrix):
    
    predicted_arc_values = y_pred[:, :num_nodes * num_nodes]
    reshaped_values = tf.reshape(predicted_arc_values, (-1, num_nodes, num_nodes))

    # Path cost
    cost_term = tf.reduce_sum(flattened_cost_matrix * predicted_arc_values)

    # Outgoing edge constraint
    outgoing_sums = tf.reduce_sum(reshaped_values, axis=2)
    outgoing_edge_penalty = tf.reduce_sum(tf.square(outgoing_sums - 1))

    # Incoming edge constraint
    incoming_sums = tf.reduce_sum(reshaped_values, axis=1)
    incoming_edge_penalty = tf.reduce_sum(tf.square(incoming_sums - 1))

    # Binary values constraint
    binary_penalty = tf.reduce_sum(predicted_arc_values * (1 - predicted_arc_values))

    loss = cost_term + 100 * outgoing_edge_penalty + 100 * incoming_edge_penalty + 10 * binary_penalty
    return loss

# Define offline training model
offline_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(num_nodes * num_nodes,)),
        tf.keras.layers.Dense(num_nodes * num_nodes, activation='sigmoid')
])

# Compile offline model
offline_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                          loss=lambda y_true, y_pred: offline_loss(y_true, y_pred, num_nodes, flattened_cost_matrix))

# Train offline model with early stopping
offline_input = np.expand_dims(flattened_cost_matrix, axis=0)
offline_model.fit(
        offline_input, 
        offline_input, 
        epochs=500, 
        verbose=1, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)]
    )

# Save pre-trained model
offline_model.save('../models/offline_model.keras')

Fine-tune pretrained model for specific origin and destination

In [ ]:
import tensorflow as tf
import numpy as np

origin = 0
destination = 3

# Load pre-trained offline model
offline_model = tf.keras.models.load_model('../models/offline_model.keras', safe_mode=False,
                                           custom_objects={'offline_loss': offline_loss})

# Placeholder for cost matrix and input vector creation
C = cost_matrix
n = C.shape[0]
C_flat = C.flatten()
C_flat[np.isinf(C_flat)] = 1e6

# Create input vector with source and destination nodes
def create_input_vector(C_flat, origin, destination, n):
    origin_vector = np.zeros(n)
    origin_vector[origin] = 1
    destination_vector = np.zeros(n)
    destination_vector[destination] = 1
    return np.concatenate([C_flat, origin_vector, destination_vector])

# Fine-tuning for specific source and destination nodes
def energy_loss_with_input_vectors(y_true, y_pred):
    """
    Computes the loss for a predicted path in a graph with specified input vectors.

    This function is similar to the offline loss, but it takes into account the input vectors
    for the origin and destination nodes. It applies penalties to deviations from the
    constraints for path cost, outgoing edges, incoming edges, binary values, and the
    source and destination nodes.

    Args:
        y_true: Ground truth values (not utilized in this computation).
        y_pred: Predicted values, containing arc values, origin vector, and destination vector.

    Returns:
        tf.Tensor: The computed loss value as a TensorFlow tensor.
    """
    predicted_arcs = y_pred[:, :n * n]
    origin_vec = y_pred[:, n * n:n * n + n]
    destination_vec = y_pred[:, n * n + n:]

    # Path cost
    path_cost = tf.reduce_sum(C_flat * predicted_arcs)

    # Outgoing and Incoming edge constraints
    reshaped_arcs = tf.reshape(predicted_arcs, (-1, n, n))
    outgoing_edges = tf.reduce_sum(reshaped_arcs, axis=2) - 1
    incoming_edges = tf.reduce_sum(reshaped_arcs, axis=1) - 1
    edge_constraints = tf.reduce_sum(tf.square(outgoing_edges)) + tf.reduce_sum(tf.square(incoming_edges))

    # Binary values constraint
    binary_constraint = tf.reduce_sum(predicted_arcs * (1 - predicted_arcs))

    # Source and Destination constraints
    source_index = tf.argmax(origin_vec, axis=1)
    source_outflow = tf.gather(reshaped_arcs, source_index, batch_dims=1)
    source_constraint = tf.reduce_sum(source_outflow, axis=1) - 1

    destination_index = tf.argmax(destination_vec, axis=1)
    destination_inflow = tf.gather(tf.transpose(reshaped_arcs, perm=[0, 2, 1]), destination_index, batch_dims=1)
    destination_constraint = tf.reduce_sum(destination_inflow, axis=1) - 1

    # Compute total loss
    loss = (path_cost 
            + 10 * edge_constraints 
            + 10 * binary_constraint 
            + 100 * tf.reduce_sum(tf.square(source_constraint)) 
            + 100 * tf.reduce_sum(tf.square(destination_constraint)))
    
    return loss

# Define full model for fine-tuning
input_dim = len(C_flat) + 2 * n
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_dim,)),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])

# Load weights from offline model and adjust
offline_weights = offline_model.layers[0].get_weights()
offline_kernel, offline_bias = offline_weights

# Adjust weights to match the new input dimension
new_kernel = np.zeros((input_dim, input_dim))  # Create new kernel with the correct size
new_bias = np.zeros(input_dim)  # Adjust bias size

# Copy offline weights for the C_flat portion
new_kernel[:len(C_flat), :len(C_flat)] = offline_kernel
new_bias[:len(C_flat)] = offline_bias

# Set adjusted weights into the fine-tuning model
model.layers[0].set_weights([new_kernel, new_bias])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=energy_loss_with_input_vectors)

# Create input vector
input_vector = create_input_vector(C_flat, origin, destination, n)
input_data = np.expand_dims(input_vector, axis=0)

# Fine-tune the model
model.fit(input_data, input_data, epochs=500, verbose=1)

# Predictions and results
predictions = model.predict(input_data)[0]
arc_values = predictions[:n * n]
arc_matrix = arc_values.reshape((n, n)) > 0.5

print("Matriz de arcos seleccionados:")
print(arc_matrix.astype(int))